# Voxel-to-voxel correlation maps
### This notebook is used to build the connectivity matrix required by CBP Tools

## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [194]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

### <font color=009DD2> I Configuration & parameters </font>

In [195]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

# Parameters (maybe we should move them to config?)
signal='raw'
seed_names=['C3'] # Define seed names
target_name=['brain'] # Define structure to use as target (brain or spinal cord)
ktot = 9

### <font color=009DD2> II Select files </font>

In [196]:
# /!\ Important notes: 
# Different from ai-based correlation analyses, no _ai suffix appended here for the timeseries txt files

# One target per subject
data_target=[];ts_target_txt=[]
for subject_name in config['list_subjects']:
    data_target.append(config['main_dir'] + '1_nilearn-based/2_func_in_template/sub-'+ subject_name + '/brain/sub-' + subject_name + '_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz')
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/sub-' + subject_name + '_' + target_name[0] + '_all_timeseries')
mask_target=config['main_dir'] + config['masks'][target_name[0].split('_')[0]]

# But possible to have multiple seeds (so create a dictionary to store list of each seed)
data_seed={}; ts_seed_txt={}; mask_seed={}
for seed_name in seed_names:
    data_seed[seed_name] = []
    ts_seed_txt[seed_name] = []
    
    # For ICA-based seeds
    mask_seed[seed_name] = glob.glob(config['main_dir'] + config['data']['ica']['spinalcord_dir'] + 'k_' + str(ktot) + '/' + 'comp_bin/' + config['data']['ica']['tag_filename'] + '*' + seed_name + '*.gz')[0]
    for subject_name in config['list_subjects']:
        data_seed[seed_name].append(config['main_dir'] + '1_nilearn-based/2_func_in_template/sub-'+ subject_name + '/spinalcord/sub-' + subject_name + '_task-rest_bold_stc_sc_moco_denoised_inTemplate.nii.gz')
        ts_seed_txt[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icas_k' + str(ktot) +'/'+seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries')
    
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icas_k' + str(ktot) + '/' +seed_name+'/'+target_name[0]+'_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icas_k' + str(ktot) +'/'+seed_name+'/'+target_name[0]+'_fc_maps/')

# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all')
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/') # folder to store timeseries extraction

### <font color=009DD2> III Initialize class based on this </font>

In [197]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_name) # initialize the function

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction


### <font color=009DD2> I. Time series extraction - Seed </font>

In [73]:
# Dictionary with seed names
# Contains one array per subject
seed_timeseries={};seed_timeseries_mean={};seed_timeseries_pc1={};
for seed_name in seed_names:
    seed_timeseries[seed_name]=[]
    seed_timeseries_mean[seed_name]=[]
    seed_timeseries_pc1[seed_name]=[]
    (seed_timeseries[seed_name],seed_timeseries_mean[seed_name],seed_timeseries_pc1[seed_name])=seed2voxels.extract_data(img=data_seed[seed_name], mask=mask_seed[seed_name],timeseries_txt=ts_seed_txt[seed_name],run="extract",n_jobs=8) # Load iCAP timecourses as seed

### <font color=009DD2> II. Time series extraction - Target </font>

In [199]:
(target_timeseries,target_timeseries_mean,target_timeseries_pc1)=seed2voxels.extract_data(img=data_target, smoothing_fwhm=[6,6,6], mask=mask_target, timeseries_txt=ts_target_txt,
                run="extract",n_jobs=8) # Load voxelwise brain signals as target


## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

### <font color=009DD2> Loop through voxels to compute correlation maps </font>

In [200]:
import numpy as np

output_img={};
for seed_name in seed_names: # For each seed
    output_img[seed_name]=[]
    seed = np.array(seed_timeseries[seed_name])
    # Loop through voxels
    for voxel in range(seed.shape[2]):
        seed_voxel = seed[:,:,voxel]
        output_img[seed_name].append(seed2voxels.correlation_maps(seed_voxel,target_timeseries,mask=mask_target,output_img=None,Fisher=True,save_maps=False,n_jobs=8))
        

### <font color=009DD2> Prepare CBP-style matrix for each subject </font>

In [170]:
np.squeeze(np.array(output_img['C3'])[:,0,:,]).shape # To check size

(4300, 44238)

In [202]:
for sub_ix,sub in enumerate(seed2voxels.subject_names):
    np.save(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icas_k' + str(ktot) +'/C3/' +target_name[0]+'_fc_maps/sub-' + sub + '_C3_voxels2brain',np.squeeze(np.array(output_img['C3'])[:,sub_ix,:,]))

### <font color=009DD2> Prepare seed_coordinates array for CBP </font>

In [175]:
import nibabel as nib
mask_seed_img = nib.load(mask_seed['C3'])
mask_seed_data = mask_seed_img.get_fdata()

In [185]:
seed_coordinates = np.array(np.where(mask_seed_data))
# 3 (x,y,z) x number of positive elements

In [201]:
np.save(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icas_k' + str(ktot) +'/C3/seed_coordinates',seed_coordinates.T)